Ignis: Latent Dirichlet Allocation
============

In [1]:
import ignis

In [2]:
# Python setup: Note that the `PYTHONHASHSEED` environmental variable needs to be set *before* the Python kernel is intialised --
# We only print it here for easy post-hoc reference.
import os
os.environ.get("PYTHONHASHSEED")

'11399'

In [3]:
# Jupyter notebook setup
import ipywidgets as widgets
from IPython.core.display import display, HTML

# Custom styling:
# - Prevent vertical scrollbars in output subareas
# - Resize to fit pyLDAvis visualisations without causing other cells to overflow
style = """
<style>
   .jupyter-widgets-output-area .output_scroll {
        height: unset !important;
        border-radius: unset !important;
        -webkit-box-shadow: unset !important;
        box-shadow: unset !important;
    }
    .jupyter-widgets-output-area  {
        height: auto !important;
    }
</style>
<style>
    #notebook-container { width: 1370px !important; }
    div.output_area { width: unset !important; }
</style>
"""
display(HTML(style))

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

In [4]:
corpus = ignis.load_corpus("bbc-full.corpus")

In [5]:
model_options = {
    "k": 6,
    "term_weighting": "idf",
    "until_max_ll": False,
    "verbose": True,
    "seed": 7157,
}
vis_options = {"verbose": True}
results = ignis.train_model(
    corpus,
    model_type="tp_lda",
    model_options=model_options,
    vis_type="pyldavis",
    vis_options=vis_options,
)

Training LDA model on 2122 documents:
{'term_weighting': 'idf', 'k': 6, 'seed': 7157, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:14<00:00, 68.16it/s, Log-likelihood=-21.11796]

Model training complete. (14.675s)
Preparing LDA visualisation

 . . . . . .distinctiveness/saliency calcs 6.338s
??? 0.007s
 . . . . .loglift stuff 6.237s
 . .concats 2.040s
 Done. (16.472s)


In [ ]:
results.save("bbc-full.aurum")
results = ignis.load_results("bbc-full.aurum")

In [ ]:
results.init_labeller("tomotopy", verbose=True)

Graphical visualisation
--------

In [6]:
results.show_visualisation()

In [ ]:
# results.export_visualisation("bbc_results")

Exploring the top words and suggested labels for each topic
------

In [ ]:
results.nb_show_topics(top_labels=15, top_words=15)

## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.

In [ ]:
# Topic 4 appears to be related to the entertainment industry
results.nb_show_topic_documents(topic_id=4, within_top_n=2)

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [ ]:
# Try zooming in on Topic 4
entertainment_slice = results.slice_by_topic(topic_id=4, within_top_n=2)
entertainment_model = results.retrain_model(corpus_slice=entertainment_slice)

In [ ]:
entertainment_model.show_visualisation()

In [ ]:
# entertainment_model.export_visualisation("bbc_results_2")

The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [ ]:
model_options = {"seed": 1234}
entertainment_model_2 = entertainment_model.retrain_model(model_options=model_options)
entertainment_model_2.show_visualisation()

In [ ]:
# entertainment_model_2.export_visualisation("bbc_results_2a")

Alternatively, we can slice the corpus by explicitly searching for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [ ]:
# Topic that seems to deal with films
film_topic = entertainment_model_2.slice_by_topic(topic_id=2, within_top_n=2)

# Tokens that are related to films
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_topic.concat(by_token_slice)

model_options = {"iterations": 2000, "k": 5}
concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

In [ ]:
# concat_model.export_visualisation("bbc_results_3")

In [ ]:
# Topic 3 appears to be related to film awards
concat_model.nb_show_topic_documents(topic_id=3, within_top_n=2)